<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


## Automatic Face Image Annotation & Masked Photo Generation ##

In the previous notebook, we prepared the unmasked photos in the dataset. In this notebook we will look at pre-processing the images for our masked photo dataset.

Firstly we will identify the bounding boxes for the faces as with the unmasked photos but additionally we will then also place a series of masked into those faces thus making them masked photos.

### Import required libraries

In [1]:
!pip install lxml
!pip install opencv-python
!pip install cmake
!pip install dlib
!pip install face_recognition

import os
import cv2
import numpy as np
import glob

import shutil
import sys
import random
import face_recognition

from lxml import etree as ET
from PIL import Image, ImageFile

     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=a382cd83f1e9697fb3fb3e88a435696efea7dd1bfd80c25d155456defff3156f
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


### Prepare dataset ###

#### Connect to Google Drive


In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Download faces dataset and extract to folder

In [20]:
!mkdir -p /content/dataset/masked

!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_01.zip

#Use 7zip to extract only the image files from our archive
!7z e 1k_faces_01.zip -o/content/dataset/masked *.jpg *.png *.jpeg -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 23768454 bytes (23 MiB)

Extracting archive: 1k_faces_01.zip
--
Path = 1k_faces_01.zip
Type = zip
Physical Size = 23768454

  0%    
Would you like to replace the existing file:
  Path:     /content/dataset/masked/00SD82OK2A.jpg
  Size:     25227 bytes (25 KiB)
  Modified: 2019-04-19 13:58:00
with the file from archive:
  Path:     1k_faces_01/00SD82OK2A.jpg
  Size:     25227 bytes (25 KiB)
  Modified: 2019-04-19 12:57:59
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% - 1k_faces_01/00SD82OK2A.jpg                                  39% 388 - 1k_faces_01/DKX37P12IG.jpg             

#### Set image directory ####

In [0]:
dataset_dir = "dataset/masked"

#### Download required dependency

In [5]:
%cd /content/
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml

/content
--2020-06-05 12:58:48--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54039 (53K) [text/plain]
Saving to: ‘lbpcascade_frontalface_improved.xml’

lbpcascade_frontalf 100%[===================>]  52.77K  --.-KB/s    in 0.008s  

2020-06-05 12:58:48 (6.52 MB/s) - ‘lbpcascade_frontalface_improved.xml’ saved [54039/54039]



In [6]:
path = os.getcwd()
filename = path + "/" + dataset_dir
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')
dataset_labels_folder = dataset_dir + "-labels"
dataset_labels_path = os.getcwd()+ "/"+ dataset_labels_folder 
print(dataset_labels_path)
os.mkdir(dataset_labels_folder)


/content/dataset/masked-labels


In [0]:
for subdir, dirs, files in os.walk(filename):
    for file in files:

        
        img_path=os.path.join(subdir, file)
        img_name=os.path.basename(img_path)

        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):

            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,w,bpp = np.shape(img)
            imgp = Image.open(img_path)
#            b, landmarks = detect_faces(imgp)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)


#            g=show_bboxes(imgp, b, landmarks)
            #print(len(faces))
            if len(faces) == 1 :
                try :

                    for bounding_boxes in faces:
                        face = img[int(bounding_boxes[1]):int(bounding_boxes[3]),
                        int(bounding_boxes[0]):int(bounding_boxes[2])]

                        
                        subdir_path, subdir_name = os.path.split(subdir)

                        root = ET.Element("annotation",verified="yes")
                        ET.SubElement(root, "folder").text=filename

                        ET.SubElement(root, "filename").text = img_name
                        ET.SubElement(root, "path").text = img_path

                        source=ET.SubElement(root, "source")
                        ET.SubElement(source, "database").text = "unknown"

                        size=ET.SubElement(root, "size")
                        ET.SubElement(size, "width").text = str(w)
                        ET.SubElement(size, "height").text = str(h)
                        ET.SubElement(size, "depth").text = str(bpp)

                        ET.SubElement(root, "segmented").text = "0"

                        obj=ET.SubElement(root, "object")
                        ET.SubElement(obj, "name").text = subdir_name
                        ET.SubElement(obj, "pose").text = "Unspecified"
                        ET.SubElement(obj, "truncated").text = "0"
                        ET.SubElement(obj, "difficult").text = "0"

                        box=ET.SubElement(obj, "bndbox")
                        ET.SubElement(box, "xmin").text = str(int(bounding_boxes[0]))
                        ET.SubElement(box, "ymin").text = str(int(bounding_boxes[1]))
                        ET.SubElement(box, "xmax").text = str(int(bounding_boxes[2])+int(bounding_boxes[1]))
                        ET.SubElement(box, "ymax").text = str(int(bounding_boxes[3]))

                        tree = ET.ElementTree(root)
                        tree.write(os.path.join(dataset_labels_path, os.path.splitext(img_name)[0] + '.xml'))
                        #tree.write(os.path.join(filename +"-labels", os.path.splitext(img_name)[0] + '.xml'))
                        
                except RuntimeError :
                    with open("delete.txt", "a") as myfile:
                        myfile.write(img_path+"\n")
                	#print(img_path)

                else :
                    with open("delete.txt", "a") as myfile:
                            myfile.write(img_path+"\n")
                    #print('no face')

## Generate Masked Photos

### Download required dependency and extract to folder

In [8]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/mask_photos.tar.xz
!mkdir -p training_demo
!tar -xf mask_photos.tar.xz -C training_demo

--2020-06-05 12:59:12--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/mask_photos.tar.xz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1652232 (1.6M) [application/octet-stream]
Saving to: ‘mask_photos.tar.xz’

mask_photos.tar.xz  100%[===================>]   1.58M  --.-KB/s    in 0.06s   

2020-06-05 12:59:13 (26.2 MB/s) - ‘mask_photos.tar.xz’ saved [1652232/1652232]



### Define environment

In [0]:
input_folder = 'dataset/masked'
mask_folder = 'training_demo/mask'
model = "hog" # Select facial recognition model; Available options areh 'hog' or 'cnn'. Hog is faster, cnn is more accurate.

num_generated_files = 0
input_folder_list = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
mask_folder_list = [os.path.join(mask_folder, f) for f in os.listdir(mask_folder) if os.path.isfile(os.path.join(mask_folder, f))]

input_folder_path = os.path.join(os.getcwd(), input_folder)
mask_folder_path = os.path.join(os.getcwd(), mask_folder)

### Helper code to place masks on the faces

In [0]:
def save(face_image_file, current_image_file):
    face_image_file.save(current_image_file)
    print(f'Save to {current_image_file}')
    

def create_mask(current_image_file, current_mask_file, model):
    show = False

    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin')
    face_image_file: ImageFile = None
    mask_image_file: ImageFile = None

    face_image_np = face_recognition.load_image_file(current_image_file)
    face_locations = face_recognition.face_locations(
        face_image_np, model=model)
    face_landmarks = face_recognition.face_landmarks(
        face_image_np, face_locations)
    face_image_file = Image.fromarray(face_image_np)
    mask_image_file = Image.open(current_mask_file)

    found_face = False
    for face_landmark in face_landmarks:

        # check whether facial features meet requirement
        skip = False
        for facial_feature in KEY_FACIAL_FEATURES:
            if facial_feature not in face_landmark:
                skip = True
                break
            if skip:
                continue

            # mask face
            found_face = True
            place_mask(face_landmark, mask_image_file, face_image_file)

    if found_face:
        if show:
            face_image_file.show()

        # save
        save(face_image_file,current_image_file)
    else:
        print('Found no face.')


def place_mask(face_landmark: dict, mask_image_file, face_image_file):
    nose_bridge = face_landmark['nose_bridge']
    nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
    nose_v = np.array(nose_point)

    chin = face_landmark['chin']
    chin_len = len(chin)
    chin_bottom_point = chin[chin_len // 2]
    chin_bottom_v = np.array(chin_bottom_point)
    chin_left_point = chin[chin_len // 8]
    chin_right_point = chin[chin_len * 7 // 8]

    # split mask and resize
    width = mask_image_file.width
    height = mask_image_file.height
    width_ratio = 1.2
    new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

    # left
    mask_left_img = mask_image_file.crop((0, 0, width // 2, height))
    mask_left_width = get_distance_from_point_to_line(
        chin_left_point, nose_point, chin_bottom_point)
    mask_left_width = int(mask_left_width * width_ratio)
    mask_left_img = mask_left_img.resize((mask_left_width, new_height))

    # right
    mask_right_img = mask_image_file.crop((width // 2, 0, width, height))
    mask_right_width = get_distance_from_point_to_line(
        chin_right_point, nose_point, chin_bottom_point)
    mask_right_width = int(mask_right_width * width_ratio)
    mask_right_img = mask_right_img.resize((mask_right_width, new_height))

    # merge mask
    size = (mask_left_img.width + mask_right_img.width, new_height)
    mask_img = Image.new('RGBA', size)
    mask_img.paste(mask_left_img, (0, 0), mask_left_img)
    mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

    # rotate mask
    angle = np.arctan2(
        chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
    rotatedmask_image_file = mask_img.rotate(angle, expand=True)

    # calculate mask location
    center_x = (nose_point[0] + chin_bottom_point[0]) // 2
    center_y = (nose_point[1] + chin_bottom_point[1]) // 2

    offset = mask_img.width // 2 - mask_left_img.width
    radian = angle * np.pi / 180
    box_x = center_x + int(offset * np.cos(radian)) - \
        rotatedmask_image_file.width // 2
    box_y = center_y + int(offset * np.sin(radian)) - \
        rotatedmask_image_file.height // 2

    # add mask
    face_image_file.paste(mask_img, (box_x, box_y), mask_img)


def get_distance_from_point_to_line(point, line_point1, line_point2):
    distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                      (line_point1[0] - line_point2[0]) * point[1] +
                      (line_point2[0] - line_point1[0]) * line_point1[1] +
                      (line_point1[1] - line_point2[1]) * line_point1[0]) / \
        np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
    return int(distance)


#### We loop through all the files in our input directory, place masks on the faces and the save them in the output directory

In [15]:
for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
     random_mask_file = os.getcwd() + "/" + random_mask_file
     
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg', '.xml')): 
      # get the next image from the image folder
      current_image_file = os.getcwd() + "/" + input_folder_list[i]
      create_mask(current_image_file, random_mask_file, model)


Save to /content/dataset/masked/WHHC578I6K.jpg
Save to /content/dataset/masked/XCYCNCMDIF.jpg
Save to /content/dataset/masked/GAW0VO7OTB.jpg
Found no face.
Save to /content/dataset/masked/0PKNKA4WR6.jpg
Save to /content/dataset/masked/E4ZTFZD4MW.jpg
Save to /content/dataset/masked/9NU2BBTWZW.jpg
Save to /content/dataset/masked/PEWEFU6VFO.jpg
Found no face.
Save to /content/dataset/masked/HLNGKGIVQ5.jpg
Save to /content/dataset/masked/YRLZFEQDG9.jpg
Save to /content/dataset/masked/3QXE62VWSO.jpg
Save to /content/dataset/masked/X76STDZ3DI.jpg
Save to /content/dataset/masked/PTE1ZN0MZ2.jpg
Found no face.
Save to /content/dataset/masked/5OYYEYFED0.jpg
Save to /content/dataset/masked/L41K4XO4F3.jpg
Save to /content/dataset/masked/9XGBZIBQZE.jpg
Found no face.
Save to /content/dataset/masked/32SS01SQ0U.jpg


UnidentifiedImageError: ignored

#### Save the generated images and archive them

In [0]:
dataset_labels_folder = input_folder_path + "-labels"
source1 = input_folder_path + "/"
source2 = dataset_labels_folder  + "/"

destination = os.getcwd() + "/training_demo"+ "/images"

if not os.path.exists(destination):
    os.makedirs(destination)


def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        shutil.move(source+f, destination)


move_files(source1,destination)
move_files(source2,destination)

# deletes source directories, use as needed
#os.rmdir(source2)
#os.rmdir(source1)


### Compress Dataset (Optional) ###

Follow this step if you wish to archive your dataset for future use

In [0]:
!zip -r masked-dataset.zip /content/training_demo/images

  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/QSU5IB5K0V.xml (deflated 46%)
  adding: content/training_demo/images/JLVFL67WK4.jpg (deflated 0%)
  adding: content/training_demo/images/LO768VIZ0S.jpg (deflated 0%)
  adding: content/training_demo/images/PJC0D2WWX4.xml (deflated 46%)
  adding: content/training_demo/images/BO9IP5AS9Q.jpg (deflated 0%)
  adding: content/training_demo/images/2GRFXVBB36.xml (deflated 46%)
  adding: content/training_demo/images/53FY0ZMLN7.jpg (deflated 0%)
  adding: content/training_demo/images/F0ZN6JPDPY.jpg (deflated 0%)
  adding: content/training_demo/images/YN42WR2FOD.xml (deflated 46%)
  adding: content/training_demo/images/O6QVGFBMVW.xml (deflated 46%)
  adding: content/training_demo/images/OW1NR9IR29.xml (deflated 46%)
  adding: content/training_demo/images/9LP3E3GMYW.xml (deflated 46%)
  adding: content/training_demo/images/US3AALGVJJ.jpg (deflated 1%)
  adding: content/training_demo/images/0U3HEWUQ72.jpg (de

#### Save archive to Google Drive

In [0]:
!gsutil cp masked-dataset.zip /content/drive/My\ Drive/Datasets/

Copying file://masked-dataset.zip...
/ [1 files][105.9 MiB/105.9 MiB]                                                
Operation completed over 1 objects/105.9 MiB.                                    
